In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tqdm

In [22]:
tf.__path__

['/opt/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/tensorflow',
 '/opt/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/api/_v2',
 '/opt/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/tensorboard/summary/_tf',
 '/opt/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/tensorflow',
 '/opt/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/tensorflow/_api/v2']

In [2]:
def resize_fun(x):
    return tf.image.resize_with_crop_or_pad(x, target_height=640,target_width=640)

In [3]:
def create_gen(df, directory='../ava-data/AVA_dataset/images_ext/images/'):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator()
    return datagen.flow_from_dataframe(df, 
                                       directory=directory, 
                                       x_col='data', y_col='score', class_mode='raw', 
                                       target_size=(640,640), resizing_function=resize_fun)

In [4]:
tf.__version__

'2.2.0'

In [5]:
ava = pd.read_csv('../ava-data/AVA_dataset/AVA.txt', sep=' ', names=['index', 'ID', *['r'+str(x) for x in range(1,11)], 't1', 't2', 'CID'])

In [6]:
test_df = pd.DataFrame(columns=['data', 'score'])
with open('../ava-data/AVA_dataset/aesthetics_image_lists/generic_test.jpgl') as file:
    for line in file.readlines():
        try:
            fID = line.strip()
            #print(fID),print(type(fID))
            raw_scores = ava.loc[ava.ID == int(fID), 'r1':'r10'].values
            score = raw_scores.dot(np.arange(1,11))/raw_scores.sum()
            test_df = test_df.append({'data':str(fID)+'.jpg', 'score':np.float32(score[0])}, ignore_index=True)
        except:
            continue

In [7]:
train_df = pd.DataFrame(columns=['data', 'score'])
with open('../ava-data/AVA_dataset/aesthetics_image_lists/generic_ss_train.jpgl') as file:
    for line in file.readlines():
        try:
            fID = line.strip()
            #print(fID),print(type(fID))
            raw_scores = ava.loc[ava.ID == int(fID), 'r1':'r10'].values
            score = raw_scores.dot(np.arange(1,11))/raw_scores.sum()
            train_df = train_df.append({'data':str(fID)+'.jpg', 'score':np.float32(score[0])}, ignore_index=True)
        except:
            print(fID, score)

954752 []
327070 []
328020 []
327530 []
11872 []


In [8]:
val_df = test_df.sample(random_state=2, n=500)

In [9]:
training_gen = create_gen(train_df)

Found 2494 validated image filenames.


/opt/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:280: UserWarning: Found 1 invalid image filename(s) in x_col="data". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [10]:
val_gen = create_gen(val_df)

Found 500 validated image filenames.


In [11]:
test_im, test_label = next(training_gen)

In [21]:
import sys
sys.getsizeof(test_im)

157286544

In [12]:
MNv2_base = tf.keras.applications.MobileNetV2(input_shape=(640, 640, 3), include_top=False, weights=None)

In [13]:
MNv2_base_batch = MNv2_base(test_im)

In [15]:
MNv2_avg_pool = tf.keras.layers.GlobalAveragePooling2D()
MNv2_avg_pool_batch = MNv2_avg_pool(MNv2_base_batch)

In [16]:
MNv2_dense = tf.keras.layers.Dense(1)
MNv2_dense_batch = MNv2_dense(MNv2_avg_pool_batch)

In [17]:
MNv2_model = tf.keras.models.Sequential([MNv2_base, MNv2_avg_pool, MNv2_dense])

In [18]:
initial_learning_rate = 0.45
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.98,
    staircase=True)

optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)
tqdm_callback = tfa.callbacks.TQDMProgressBar()

In [19]:
MNv2_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['MSE', 'MAE'])

In [99]:
loss, MSE, MAE = MNv2_model.evaluate(val_gen, steps=20)

20/20 [==============================] - 66s 3s/step - loss: 29.5837 - MSE: 29.5837 - MAE: 5.3917


In [20]:
MNv2_model.fit(x=training_gen, epochs=5, validation_data=val_gen, verbose=1)

Epoch 1/5


KeyboardInterrupt: 